In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.prompts import PromptTemplate

In [4]:
template = "{country}의 수도는 어디인가요?"

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [5]:
prompt = prompt_template.format(country='한국')
prompt

'한국의 수도는 어디인가요?'

In [6]:
prompt2 = prompt_template.format(country='네덜란드')
prompt2

'네덜란드의 수도는 어디인가요?'

**Model(LLM)**

In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-3.5-turbo',
    max_tokens=2048,
    temperature=0.1,
)

In [10]:
from langchain_core.output_parsers import StrOutputParser

output = StrOutputParser()

**LCEL로 chain 생성**

    - chain = prompt | model | output

In [14]:
template = """
당신은 친절하게 답변해주는 친절한 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해주세요.
답변은 항상 한글로 작성해 주세요.

질문 :
{question}에 대해 설명해 주세요.

FORMAT :
- 개요:
- 예시:
- 출처:
"""

template2 = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

In [15]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['question'], template='\n당신은 친절하게 답변해주는 친절한 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해주세요.\n답변은 항상 한글로 작성해 주세요.\n\n질문 :\n{question}에 대해 설명해 주세요.\n\nFORMAT :\n- 개요:\n- 예시:\n- 출처:\n')

In [16]:
prompt2 = PromptTemplate.from_template(template2)
prompt2

PromptTemplate(input_variables=['question'], template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.\n\n상황:\n{question}\n\nFORMAT:\n- 영어 회화:\n- 한글 해석:\n')

In [17]:
model = ChatOpenAI(model='gpt-3.5-turbo')
output_parser = StrOutputParser()

In [18]:
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['question'], template='\n당신은 친절하게 답변해주는 친절한 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해주세요.\n답변은 항상 한글로 작성해 주세요.\n\n질문 :\n{question}에 대해 설명해 주세요.\n\nFORMAT :\n- 개요:\n- 예시:\n- 출처:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x147e819c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x147e832b0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [19]:
chain2= prompt2 | model | output_parser
chain2

PromptTemplate(input_variables=['question'], template='\n당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.\n\n상황:\n{question}\n\nFORMAT:\n- 영어 회화:\n- 한글 해석:\n')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x147e819c0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x147e832b0>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [20]:
print(chain.invoke({'question' : 'python 사용 방법'}))

개요: 파이썬은 인터프리터 언어로 간단하고 읽기 쉬운 문법을 가지고 있어 많은 사람들이 사용하는 프로그래밍 언어입니다. 

예시: 
1. 파이썬 인터프리터를 다운로드하여 설치합니다.
2. 텍스트 에디터나 통합 개발 환경(IDE)를 이용하여 파이썬 코드를 작성합니다.
3. 작성한 코드를 실행하여 원하는 결과를 얻을 수 있습니다.

출처: 본 내용은 제가 작성한 것이며, 파이썬 공식 문서 및 다양한 온라인 자료를 참고하였습니다.


In [23]:
print(chain2.invoke({'question' : '피자 주문'}))

- 영어 회화: 
Customer: Hi, I'd like to place an order for delivery.
Employee: Of course! What would you like to order?
Customer: I'd like a large pepperoni pizza, please.
Employee: Any drinks or sides with that?
Customer: Can I get a 2-liter bottle of Coke and an order of garlic knots?
Employee: Sure thing. Your total comes to $25. Will that be cash or card?
Customer: I'll pay with my card, please.

- 한글 해석:
고객: 안녕하세요, 배달 주문을 하고 싶은데요.
직원: 물론이죠! 무엇을 주문하시겠어요?
고객: 큰 페퍼로니 피자 한 판 주문하고 싶어요.
직원: 그 외에 음료나 사이드 메뉴는 더 필요하신가요?
고객: 2리터 콜라 한 병과 마늘 노트 주문 가능할까요?
직원: 물론이죠. 총 금액은 $25 입니다. 현금으로 결제하시겠습니까, 아니면 카드로 결제하시겠습니까?
고객: 카드로 결제할게요.


**전체 파이프라인**

    (1) 원하는 주제에 대한 사용자 입력 {'topic' : '양자역학'}
    (2) `prompt` 컴포넌트는 사용자 입력을 받아 topic을 사용해 프롬프트를 구성해 PromptValue 생성
    (3) model 컴포넌트는 생성된 프롬프트를 가져와 OpenAI LLM 모델에 평가를 위해 전달함. 모델에서 생성된 출력은 `ChatMessage` 객체
    (4) 마지막 `output_parser` 컴포넌트는 `ChatMessage`를 받아 python 문자열로 변환하고, `invoke` 메서드에서 반환됨

In [24]:
template = "{topic}에 대해 쉽게 설명해주세요"

prompt = PromptTemplate.from_template(template)
print(prompt)

input = {'topic' : '양자역학'}

prompt.invoke(input)



input_variables=['topic'] template='{topic}에 대해 쉽게 설명해주세요'


StringPromptValue(text='양자역학에 대해 쉽게 설명해주세요')

In [25]:
(prompt | model).invoke(input)

AIMessage(content='양자역학은 아주 작은 입자들인 원자나 입자들의 행동을 설명하는 물리 이론입니다. 이 이론은 입자들이 특정한 위치와 운동량을 동시에 가질 수 없다는 불확정성 원리를 중요하게 다룹니다. 또한 입자들이 원자 내에서 특정한 에너지 상태로만 존재할 수 있다는 양자화 개념을 포함하고 있습니다.\n\n양자역학은 빛이나 전자와 같은 입자들의 파동-입자 이중성, 양자역학 현상인 얽힘(entanglement), 그리고 양자계산 등 다양한 현상을 설명하는데 사용됩니다. 이론은 매우 복잡하고 추상적이지만, 많은 현대 기술과 기술의 발전에 중요한 역할을 하고 있습니다.', response_metadata={'token_usage': {'completion_tokens': 270, 'prompt_tokens': 25, 'total_tokens': 295}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [26]:
(prompt | model | output_parser).invoke(input)

'양자역학은 아주 작은 입자들인 원자나 분자 수준에서의 물리 현상을 설명하는 이론입니다. 이론은 확률적이고 불확실성이 존재하며, 입자의 위치나 운동 상태를 정확히 예측하는 것이 불가능하다는 것을 주장합니다. 대신, 입자의 상태를 확률적으로 설명하고 그런 상태에서 어떤 결과가 나올 확률을 계산합니다.\n\n양자역학은 빛이나 전자 같은 입자들이 파동의 성질도 가지고 있다는 것을 밝혀냈으며, 물리학의 다양한 현상을 설명하는 데 중요한 역할을 합니다. 또한 원자나 분자의 구조, 물질의 성질, 빛의 발생과 흡수 등을 설명하는 데 사용됩니다.양자역학은 현대 물리학에서 중요한 이론 중 하나로 인류가 우주와 물질에 대해 이해하는 데 도움을 줍니다.'